In [1]:
from langchain_community.llms.llamacpp import LlamaCpp
from transformers import AutoTokenizer
import torch

gguf_path = './Mistral-Nemo-Instruct-2407-Q8_0.gguf'
llm = LlamaCpp(
  model_path=gguf_path,
  n_gpu_layers=-1,
  temperature=0,
  n_ctx=10000,
  verbose=False
)
tokenizer = AutoTokenizer.from_pretrained('./Mistral-Nemo-Instruct-2407')


In [2]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from runnables import generate_history_passthrough_function, generate_llm_function, MistralAgentOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain.agents.output_parsers import JSONAgentOutputParser

history_passthrough = RunnablePassthrough.assign(history=generate_history_passthrough_function())
llm_runnable = RunnableLambda(generate_llm_function(tokenizer, llm))

agent = history_passthrough | llm_runnable | MistralAgentOutputParser()


In [3]:
output = agent.invoke({
  'input': 'please tell me current weather in seoul',
  'intermediate_steps': []
})

print(output)


/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


tool='get_current_weather' tool_input={'location': 'seoul', 'format': 'celsius'} log='no log'


In [4]:
from langchain.agents import AgentExecutor
from tools import get_current_weather, send_email, search_web

tools = [
  get_current_weather,
  send_email,
  search_web
]
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True, max_iterations=5, return_intermediate_steps=True)


In [11]:
output = agent_executor.invoke(
  input={
    'input': 'search about average temperature of seoul and compare to current temperature of seoul and send email about the result to aa@gmail.com.'
  }
)
output




> Entering new AgentExecutor chain...


/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


no logthe average day time temperature of seoul is 32 degree celsius

/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


no log38 degree

/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


no logemail sent successfully

/home/akswnd98/anaconda3/envs/python-3.11/lib/python3.11/site-packages/llama_cpp/llama.py:1238: RuntimeWarning: Detected duplicate leading "<s>" in prompt, this will likely reduce response quality, consider removing it...
  warnings.warn(


I have sent an email to aa@gmail.com about the temperature comparison result.

> Finished chain.


{'input': 'search about average temperature of seoul and compare to current temperature of seoul and send email about the result to aa@gmail.com.',
 'output': 'I have sent an email to aa@gmail.com about the temperature comparison result.',
 'intermediate_steps': [(AgentAction(tool='search_web', tool_input={'text': 'average temperature of seoul'}, log='no log'),
   'the average day time temperature of seoul is 32 degree celsius'),
  (AgentAction(tool='get_current_weather', tool_input={'location': 'seoul, KR', 'format': 'celsius'}, log='no log'),
   '38 degree'),
  (AgentAction(tool='send_email', tool_input={'address': 'aa@gmail.com', 'subject': 'Temperature Comparison Report', 'body': 'Current temperature in Seoul is 38 degree celsius, which is higher than the average day time temperature of 32 degree celsius.'}, log='no log'),
   'email sent successfully')]}